<a href="https://colab.research.google.com/github/solomontessema/building-ai-agents/blob/main/notebooks/6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain==1.1.0 langchain-openai==1.1.0 python-dotenv==1.1.1

In [ ]:
import smtplib
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

AGENT_EMAIL_ADDRESS = os.getenv("AGENT_EMAIL_ADDRESS")
AGENT_EMAIL_PASSWORD = os.getenv("AGENT_EMAIL_PASSWORD")
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

def send_email(to_address, subject, body, attachment_path=None):
    if not AGENT_EMAIL_ADDRESS or not AGENT_EMAIL_PASSWORD:
        raise ValueError("Missing EMAIL_ADDRESS or EMAIL_PASSWORD in environment.")

    # Create the email message
    msg = MIMEMultipart()
    msg["From"] = AGENT_EMAIL_ADDRESS
    msg["To"] = to_address
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    # Send the email
    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.ehlo()
        server.starttls()
        server.ehlo()
        server.login(AGENT_EMAIL_ADDRESS, AGENT_EMAIL_PASSWORD)
        server.send_message(msg)
        print("Email sent successfully.")


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

quote_prompt = ChatPromptTemplate.from_template(
    "Generate the best stoic quote about {topic} and explain it briefly."
)

# FIXED CHAIN:
stoic_runnable = (
    {
        # Extract "topic" from the input and send to LLM
        "body": itemgetter("topic") | quote_prompt | llm | StrOutputParser(),
        # Extract "recipient_email" directly
        "to_address": itemgetter("recipient_email"),
        # Provide a static subject
        "subject": lambda _: "Your Daily Stoic Wisdom"
    }
    | RunnableLambda(lambda x: send_email(
        to_address=x["to_address"],
        subject=x["subject"],
        body=x["body"]
    ))
)

# --- Execution ---

if __name__ == "__main__":
    input_params = {
        "topic": "patience",
        "recipient_email": "recipient_email@example.com"
    }

    result = stoic_runnable.invoke(input_params)
    print(result)